In [3]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math
import numpy as np
import re
from shapely.geometry import Polygon, LineString, Point
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import CocoDetection
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision.transforms as T
from torch.optim import SGD, Adam, Adadelta
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision import transforms
from torch.utils.data._utils.collate import default_collate
import torchvision
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.transforms import functional as F
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import random
from math import radians, cos, sin
import ast
import shutil

# 1. Unzip & Prepare Directory

In [4]:
import tarfile

tar_path = './ds2_dense.tar.gz'
extract_path = './ds2_dense/'

with tarfile.open(tar_path, "r:gz") as tar:
    tar.extractall(path=extract_path)

# fix nested folder
nested = os.path.join(extract_path, 'ds2_dense')
if os.path.exists(nested):
    for item in os.listdir(nested):
        shutil.move(os.path.join(nested, item), extract_path)
    shutil.rmtree(nested)

# 2. Load Json file

In [5]:
# 加载 JSON
with open('./ds2_dense/deepscores_train.json') as f:
    data1 = json.load(f)
with open('./ds2_dense/deepscores_test.json') as f:
    data2 = json.load(f)

# 提取图像和标注
train_images = pd.DataFrame(data1['images'])
train_annots = pd.DataFrame(data1['annotations']).T

test_images = pd.DataFrame(data2['images'])
test_annots = pd.DataFrame(data2['annotations']).T

# 3. Splitting the Images into Train and Test sets

In [6]:
image_dir = './ds2_dense/images'
train_dir = './ds2_dense/images/train'
test_dir = './ds2_dense/images/test'
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

In [7]:
# Move train images to train directory
for image_filename in train_images['filename']:
   src_path = os.path.join(image_dir, image_filename)
   dest_path = os.path.join(train_dir, image_filename)
   shutil.move(src_path, dest_path)

In [8]:
# Move train images to test directory
for image_filename in test_images['filename']:
   src_path = os.path.join(image_dir, image_filename)
   dest_path = os.path.join(test_dir, image_filename)
   shutil.move(src_path, dest_path)

# 4. Generate Label Mapping (deepscores only)
The yolo model wants labels to be zero-based indexing


In [9]:
label_dict = list(data1.values())[2]

# 先过滤出 annotation_set == 'deepscores' 的项
filtered_items = [(k, v) for k, v in label_dict.items() if v['annotation_set'] == 'deepscores']

# 用 enumerate 连续编号
cat_dict = {
    k: {
        'old_id': int(k),
        'name': v['name'],
        'label': i
    } for i, (k, v) in enumerate(filtered_items)
}

# 转为 DataFrame
df_labels = pd.DataFrame.from_dict(cat_dict, orient='index').reset_index(drop=True)

df_labels


,old_id,name,label
0,1,brace,0
1,2,ledgerLine,1
2,3,repeatDot,2
3,4,segno,3
4,5,coda,4
...,...,...,...
131,132,tuplet8,131
132,133,tuplet9,132
133,134,tupletBracket,133
134,135,staff,134


# 5. Attach Label to each obbox

In [10]:
class_mapping = dict(zip(df_labels['old_id'].astype(str), df_labels['label']))

def map_label(cat_ids):
    return max([class_mapping.get(str(cid)) for cid in cat_ids if str(cid) in class_mapping], default=None)

train_annots['label'] = train_annots['cat_id'].apply(map_label)
test_annots['label'] = test_annots['cat_id'].apply(map_label)

train_annots.head(10)

,a_bbox,o_bbox,cat_id,area,img_id,comments,label
1020,"[116.0, 139.0, 2315.0, 206.0]","[2315.0, 206.0, 2315.0, 139.0, 116.0, 139.0, 1...","[135, 208]",18945,679,instance:#000010;,134
1021,"[116.0, 309.0, 2315.0, 376.0]","[2315.0, 376.0, 2315.0, 309.0, 116.0, 309.0, 1...","[135, 208]",19223,679,instance:#000021;,134
1022,"[1880.0, 561.0, 1911.0, 564.0]","[1911.0, 564.0, 1911.0, 561.0, 1880.0, 561.0, ...","[2, 138]",120,679,instance:#000022;,1
1023,"[1883.0, 578.0, 1911.0, 580.0]","[1911.0, 580.0, 1911.0, 578.0, 1883.0, 578.0, ...","[2, 138]",27,679,instance:#000023;,1
1024,"[1827.0, 561.0, 1857.0, 564.0]","[1857.0, 564.0, 1857.0, 561.0, 1827.0, 561.0, ...","[2, 138]",112,679,instance:#000024;,1
1025,"[1827.0, 578.0, 1857.0, 580.0]","[1857.0, 580.0, 1857.0, 578.0, 1827.0, 578.0, ...","[2, 138]",32,679,instance:#000025;,1
1026,"[1773.0, 561.0, 1804.0, 564.0]","[1804.0, 564.0, 1804.0, 561.0, 1773.0, 561.0, ...","[2, 138]",120,679,instance:#000026;,1
1027,"[1773.0, 578.0, 1804.0, 580.0]","[1804.0, 580.0, 1804.0, 578.0, 1773.0, 578.0, ...","[2, 138]",71,679,instance:#000027;,1
1028,"[1718.0, 561.0, 1748.0, 564.0]","[1748.0, 564.0, 1748.0, 561.0, 1718.0, 561.0, ...","[2, 138]",112,679,instance:#000028;,1
1029,"[1718.0, 578.0, 1748.0, 580.0]","[1748.0, 580.0, 1748.0, 578.0, 1718.0, 578.0, ...","[2, 138]",84,679,instance:#000029;,1


# 6. Adjust bounding box for 0 valued w/h

In [11]:
def adjust_bbox(bbox):
    x_min, y_min, x_max, y_max = bbox
    if x_min == x_max:
        x_min -= 1
        x_max += 1
    if y_min == y_max:
        y_min -= 1
        y_max += 1
    return [x_min, y_min, x_max, y_max]

train_annots['bbox'] = train_annots['a_bbox'].apply(adjust_bbox)
test_annots['bbox'] = test_annots['a_bbox'].apply(adjust_bbox)


# 7. Merge Annotation and Image df

In [13]:
train_annots['img_id'] = train_annots['img_id'].astype(str)
test_annots['img_id'] = test_annots['img_id'].astype(str)

train_images['img_id'] = train_images['img_id'].astype(str)
test_images['img_id'] = test_images['img_id'].astype(str)


train_images.rename(columns={'id': 'img_id'}, inplace=True)
test_images.rename(columns={'id': 'img_id'}, inplace=True)

train_df = pd.merge(train_annots, train_images, on='img_id')
test_df = pd.merge(test_annots, test_images, on='img_id')

train_df.head(10)

,a_bbox,o_bbox,cat_id,area,img_id,comments,label,bbox,filename,width,height,ann_ids
0,"[116.0, 139.0, 2315.0, 206.0]","[2315.0, 206.0, 2315.0, 139.0, 116.0, 139.0, 1...","[135, 208]",18945,679,instance:#000010;,134,"[116.0, 139.0, 2315.0, 206.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."
1,"[116.0, 309.0, 2315.0, 376.0]","[2315.0, 376.0, 2315.0, 309.0, 116.0, 309.0, 1...","[135, 208]",19223,679,instance:#000021;,134,"[116.0, 309.0, 2315.0, 376.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."
2,"[1880.0, 561.0, 1911.0, 564.0]","[1911.0, 564.0, 1911.0, 561.0, 1880.0, 561.0, ...","[2, 138]",120,679,instance:#000022;,1,"[1880.0, 561.0, 1911.0, 564.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."
3,"[1883.0, 578.0, 1911.0, 580.0]","[1911.0, 580.0, 1911.0, 578.0, 1883.0, 578.0, ...","[2, 138]",27,679,instance:#000023;,1,"[1883.0, 578.0, 1911.0, 580.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."
4,"[1827.0, 561.0, 1857.0, 564.0]","[1857.0, 564.0, 1857.0, 561.0, 1827.0, 561.0, ...","[2, 138]",112,679,instance:#000024;,1,"[1827.0, 561.0, 1857.0, 564.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."
5,"[1827.0, 578.0, 1857.0, 580.0]","[1857.0, 580.0, 1857.0, 578.0, 1827.0, 578.0, ...","[2, 138]",32,679,instance:#000025;,1,"[1827.0, 578.0, 1857.0, 580.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."
6,"[1773.0, 561.0, 1804.0, 564.0]","[1804.0, 564.0, 1804.0, 561.0, 1773.0, 561.0, ...","[2, 138]",120,679,instance:#000026;,1,"[1773.0, 561.0, 1804.0, 564.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."
7,"[1773.0, 578.0, 1804.0, 580.0]","[1804.0, 580.0, 1804.0, 578.0, 1773.0, 578.0, ...","[2, 138]",71,679,instance:#000027;,1,"[1773.0, 578.0, 1804.0, 580.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."
8,"[1718.0, 561.0, 1748.0, 564.0]","[1748.0, 564.0, 1748.0, 561.0, 1718.0, 561.0, ...","[2, 138]",112,679,instance:#000028;,1,"[1718.0, 561.0, 1748.0, 564.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."
9,"[1718.0, 578.0, 1748.0, 580.0]","[1748.0, 580.0, 1748.0, 578.0, 1718.0, 578.0, ...","[2, 138]",84,679,instance:#000029;,1,"[1718.0, 578.0, 1748.0, 580.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102..."


# 8. Convert BBOX to YOLO format

In [14]:
def convert_to_yolo(bbox, width, height):
    x_min, y_min, x_max, y_max = bbox
    x_center = (x_min + x_max) / 2 / width
    y_center = (y_min + y_max) / 2 / height
    w = (x_max - x_min) / width
    h = (y_max - y_min) / height
    return [x_center, y_center, w, h]

train_df['yolo_bbox'] = train_df.apply(lambda r: convert_to_yolo(r['bbox'], r['width'], r['height']), axis=1)
test_df['yolo_bbox'] = test_df.apply(lambda r: convert_to_yolo(r['bbox'], r['width'], r['height']), axis=1)
train_df.head(10)


,a_bbox,o_bbox,cat_id,area,img_id,comments,label,bbox,filename,width,height,ann_ids,yolo_bbox
0,"[116.0, 139.0, 2315.0, 206.0]","[2315.0, 206.0, 2315.0, 139.0, 116.0, 139.0, 1...","[135, 208]",18945,679,instance:#000010;,134,"[116.0, 139.0, 2315.0, 206.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102...","[0.5, 0.05015993021227101, 0.904566022213081, ..."
1,"[116.0, 309.0, 2315.0, 376.0]","[2315.0, 376.0, 2315.0, 309.0, 116.0, 309.0, 1...","[135, 208]",19223,679,instance:#000021;,134,"[116.0, 309.0, 2315.0, 376.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102...","[0.5, 0.09959290491421925, 0.904566022213081, ..."
2,"[1880.0, 561.0, 1911.0, 564.0]","[1911.0, 564.0, 1911.0, 561.0, 1880.0, 561.0, ...","[2, 138]",120,679,instance:#000022;,1,"[1880.0, 561.0, 1911.0, 564.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102...","[0.7797202797202797, 0.16356498982262285, 0.01..."
3,"[1883.0, 578.0, 1911.0, 580.0]","[1911.0, 580.0, 1911.0, 578.0, 1883.0, 578.0, ...","[2, 138]",27,679,instance:#000023;,1,"[1883.0, 578.0, 1911.0, 580.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102...","[0.7803373097490744, 0.16836289619075312, 0.01..."
4,"[1827.0, 561.0, 1857.0, 564.0]","[1857.0, 564.0, 1857.0, 561.0, 1827.0, 561.0, ...","[2, 138]",112,679,instance:#000024;,1,"[1827.0, 561.0, 1857.0, 564.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102...","[0.7577128753599341, 0.16356498982262285, 0.01..."
5,"[1827.0, 578.0, 1857.0, 580.0]","[1857.0, 580.0, 1857.0, 578.0, 1827.0, 578.0, ...","[2, 138]",32,679,instance:#000025;,1,"[1827.0, 578.0, 1857.0, 580.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102...","[0.7577128753599341, 0.16836289619075312, 0.01..."
6,"[1773.0, 561.0, 1804.0, 564.0]","[1804.0, 564.0, 1804.0, 561.0, 1773.0, 561.0, ...","[2, 138]",120,679,instance:#000026;,1,"[1773.0, 561.0, 1804.0, 564.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102...","[0.7357054709995886, 0.16356498982262285, 0.01..."
7,"[1773.0, 578.0, 1804.0, 580.0]","[1804.0, 580.0, 1804.0, 578.0, 1773.0, 578.0, ...","[2, 138]",71,679,instance:#000027;,1,"[1773.0, 578.0, 1804.0, 580.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102...","[0.7357054709995886, 0.16836289619075312, 0.01..."
8,"[1718.0, 561.0, 1748.0, 564.0]","[1748.0, 564.0, 1748.0, 561.0, 1718.0, 561.0, ...","[2, 138]",112,679,instance:#000028;,1,"[1718.0, 561.0, 1748.0, 564.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102...","[0.7128753599341835, 0.16356498982262285, 0.01..."
9,"[1718.0, 578.0, 1748.0, 580.0]","[1748.0, 580.0, 1748.0, 578.0, 1718.0, 578.0, ...","[2, 138]",84,679,instance:#000029;,1,"[1718.0, 578.0, 1748.0, 580.0]",lg-877777775968732096-aug-gonville--page-3.png,2431,3439,"[1020, 1021, 1022, 1023, 1024, 1025, 1026, 102...","[0.7128753599341835, 0.16836289619075312, 0.01..."


# 9. Output YOLO training information as txt

In [15]:
import os

def export_yolo_txt(df, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for fname, group in df.groupby('filename'):
        txt_path = os.path.join(output_dir, os.path.splitext(fname)[0] + '.txt')
        with open(txt_path, 'w') as f:
            for _, row in group.iterrows():
                cls = row['label']
                x, y, w, h = row['yolo_bbox']
                f.write(f"{cls} {x} {y} {w} {h}\n")

export_yolo_txt(train_df, './ds2_dense/labels/train')
export_yolo_txt(test_df, './ds2_dense/labels/test')

# 10. Save Yolo Training Meta information

In [16]:
import yaml

yaml_dict = {
    'path': os.path.abspath('./ds2_dense'),
    'train': 'images/train',
    'val': 'images/test',
    'names': df_labels.set_index('label')['name'].to_dict()
}

with open('deep_scores.yaml', 'w') as f:
    yaml.dump(yaml_dict, f)

# 11. Train Yolo Model

In [17]:
from ultralytics import YOLO
model = YOLO('yolov8m.pt') 

In [18]:
# Check if CUDA is available and configure training accordingly

device = torch.device("cuda:0")

if torch.cuda.is_available():
    print('Training with GPU.')
    results = model.train(data="deep_scores.yaml", epochs=10, batch=3, imgsz=[704, 992], device=device, patience=0, deterministic=True, amp=True) # rect=True, ,[928, 1312], imgsz=[704, 992], imgsz=[1960, 2772]
    
else:
    results = model.train(data="deep_scores.yaml", imgsz=640, rect=True, epochs = 10, batch=4)  # patience=100, device="mps", deterministic=True, amp=True

Training with GPU.
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=deep_scores.yaml, epochs=10, time=None, patience=0, batch=3, imgsz=[704, 992], save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, optimize=False, int8=False, 

train: Scanning /home/user/Desktop/OMR/ds2_dense/labels/train... 1362 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1362/1362 [00:09<00:00, 140.97it/s]


train: New cache created: /home/user/Desktop/OMR/ds2_dense/labels/train.cache
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 834.9±131.4 MB/s, size: 234.8 KB)


val: Scanning /home/user/Desktop/OMR/ds2_dense/labels/test... 352 images, 0 backgrounds, 0 corrupt: 100%|██████████| 352/352 [00:02<00:00, 134.90it/s]

val: New cache created: /home/user/Desktop/OMR/ds2_dense/labels/test.cache


...

                   all        352     244335      0.564      0.249      0.294      0.206

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      10/10      9.52G      1.038     0.7161     0.8205       1692        992: 100%|██████████| 454/454 [01:07<00:00,  6.76it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 59/59 [00:11<00:00,  5.19it/s]


                   all        352     244335      0.564       0.25      0.294      0.207

10 epochs completed in 0.224 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 52.2MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 52.2MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.121 🚀 Python-3.10.16 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 16069MiB)
Model summary (fused): 92 layers, 25,918,504 parameters, 0 gradients, 79.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 59/59 [01:51<00:00,  1.89s/it]


                   all        352     244335      0.564       0.25      0.294      0.207
                 brace        177        725      0.945      0.894       0.94      0.723
            ledgerLine        342      23809          0          0          0          0
             repeatDot         69        876          1          0          0          0
                 segno         43         55      0.931      0.909      0.964      0.864
                  coda         45         49      0.803      0.665      0.747      0.342
                 clefG        313       2203      0.988      0.997      0.995      0.949
             clefCAlto         89        255      0.658      0.898      0.798      0.691
            clefCTenor         65        167      0.683      0.701      0.741       0.66
                 clefF        294       1488      0.985      0.979      0.988      0.868
                 clef8        105        305       0.89     0.0525      0.272     0.0881
                clef1